In [4]:
import cv2, dlib  # 이미지 표현 및 눈 좌표 얻어오기
import numpy as np  # 데이터 처리
from imutils import face_utils  # 얼굴 분석
from tensorflow import keras  # 모델 학습 및 테스트
from playsound import playsound  # 소리 재생
import threading  # 스레드 사용

import os # 파일 마지막꺼 출력

import asyncio#비동기 함수 사용


import time#슬립함수

# 추출할 눈 이미지 사이즈
IMG_SIZE = (34, 26)
# 눈을 감은 프레임을 세줄 변수
n_count = 0
# 경보음 다중재생 방지 변수
global is_playing
is_playing = False

# 얼굴에 68개의 점을 찍음 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 학습한 모델을 불러옴

model = keras.models.load_model('models/2018_12_17_22_58_35.h5') #원본소스
#model = keras.models.load_model('models/2022_12_04_04_04_59.h5') 
model.summary()


#절대경로찾기
from pathlib import Path
import requests

# 눈을 찾아주는 함수
def crop_eye(img, eye_points):
    
    #start = time.time()
    
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]
    
   
    #end = time.time()
    
   # print(f"{end - start:.5f} sec")

    return eye_img, eye_rect


def playSound():
    global is_playing
    t = threading.Thread(target=thread)
    # 스레드를 시작한후 join해서 동작이 멈추길 기다린 후 is_plaing을 false로 바꿔 소리 재생이 가능하도록.
    t.start()
    t.join()
    is_playing = False


def thread():
    playsound("sound.wav")
    
    
def incodevid(): #프레임수 제한할 새 비디오 경로 만들기, 사용안함

    # 웹캠의 속성 값을 받아오기
    # 정수 형태로 변환하기 위해 round
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) # 카메라에 따라 값이 정상적, 비정상적

    # fourcc 값 받아오기, *는 문자를 풀어쓰는 방식, *'DIVX' == 'D', 'I', 'V', 'X'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')

    # 1프레임과 다음 프레임 사이의 간격 설정
    delay = round(30/fps)
    
    temp_path2 = incode_path+'//output.avi'
    
    
    out = cv2.VideoWriter(incode_path+'//output.avi', fourcc, fps, (w, h))
  
    return temp_path2

 
def vid_to_img(): # 비디오 추출해어 이미지 만들기. 사용할지 말지 결정
    fps2 = int(cap.get(cv2.CAP_PROP_FPS))
               
    count = 0
    img_count=1
    while(cap.isOpened()):
        
        ret, image = cap.read()
        
        #if(int(cap.get(1)) % fps2 == 0): #앞서 불러온 fps 값을 사용하여  초마다 추출
        try:
            if(img_count%5==0): # 5프레임으로
                #cv2.imwrite(incode_path[:-4] + "/frame%d.jpg" % count, image)
                cv2.imwrite(incode_img+"//"+str(count)+".jpg", image)
                count += 1
            img_count+=1
        except:
            break
    cap.release()
    

    #경로 읽어서 파일목록 읽어들이기, 마지막파일을  file_path에 입력
#dir_path = "testvid/membervid/"


#프레임수 제한할 새 인코딩 이미지,비디오 경로 만들기
incode_path ="C://eGovFrame-3.10.0//bin//eclipse//checkupload/incode_path/"
if(os.path.isdir(incode_path)!=True): #폴더 경로 체크 . 없으면 생성
                            os.mkdir(incode_path)
incode_img ="C://eGovFrame-3.10.0//bin//eclipse//checkupload//incode_img"
if(os.path.isdir(incode_img)!=True): #폴더 경로 체크 . 없으면 생성
                            os.mkdir(incode_img)


dir_path = "C://eGovFrame-3.10.0//bin//eclipse//checkupload//membervid"
file_path=""

for (root, directories, files) in os.walk(dir_path):
    
    for file in files:
        file_path = os.path.join(root, file)
        
#         str = Path(file_path).resolve()  #파일 절대경로 
#         file_path=str
        
        
cap=cv2.VideoCapture()

temp_file_path=""
        
def vid_path():  #dir 위치 읽어와서 비디오 죄다 읽은후 마지막 비디오 반환하기
    global file_path
    global cap
    
    
    cap = cv2.VideoCapture(file_path)
                                     
    for (root, directories, files) in os.walk(dir_path): #dir_path 내의 파일중 마지막 파일을 
        for file in files:
            file_path = os.path.join(root, file) # file_path에 값 저장
#             str = Path(file_path).resolve()
#             file_path=str
    

vid_path()
# main
#cap = cv2.VideoCapture(file_path)
# cap = cv2.VideoCapture(0)

index_write=0



def gotojsp(result):
    URL = "http://localhost:8085/controller/flask/upsleepy?deepresult="+result
    response = requests.get(URL)




while(True):   
   
    vid_path()#파일경로 찾기
    time.sleep(0.1)#영상 경로 찾는것에 딜레이주기
    cap
    index_write=0
   
    if(temp_file_path!=file_path):  #로드 영상 위치가 바뀌었을 경우, 즉 영상이 새로 촬영된경우
        temp_file_path=file_path
        
       # cap=cv2.VideoCapture(incodevid()) #인코딩 된 영상으로 분석

        #vid_to_img() #영상을 이미지로. 사용안함
        vid_path()
        
        new_count = 0
        new_fps=5
        while cap.isOpened():
            
        
            new_count+=1
            ret, img_ori = cap.read()  #ret = 프레임 읽기을 성공하면 True 값 반환 #img_ori = 배열 형식의 영상 프레임 (가로 X 세로 X 3) 값 반환

            if not ret: #ret 이 false 면 탈출
                break

                
            if(new_count%new_fps==0): #프레임 5개당 하나만 분석하기. 속도를 위해
                
                img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)
                #img_ori = cv2.resize(img_ori, dsize=(320, 240))

                img = img_ori.copy()
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                faces = detector(gray)
                
                for face in faces:
                    
                    shapes = predictor(gray, face)
                    shapes = face_utils.shape_to_np(shapes)

                    eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
                    eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

                    eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
                    eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
                    eye_img_r = cv2.flip(eye_img_r, flipCode=1)

                    # cv2.imshow('l', eye_img_l)
                    # cv2.imshow('r', eye_img_r)

                    eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
                    eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

                    pred_l = model.predict(eye_input_l)
                    pred_r = model.predict(eye_input_r)

                    if pred_l < 0.1 and pred_r < 0.1:
                        n_count += 1
                    else:
                        n_count = 0
                        is_playing = False
                        # playsound("")

                        #프레임수 조절, 30프레임은 1초
                    if n_count > 20:
                        cv2.putText(img, "Wake up", (120, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                        # 스레드에서 사운드 재생
        #                 if not is_playing:
        #                     is_playing = True
        #                     t = threading.Thread(target=playSound)
        #                     t.start()

                                    
                        gotojsp(file_path)      #판정결과 졸음일경우 jsp 접속 함수 호출                     
                        n_count=0  #판정위해 추가,. 졸음 판정되면 즉시 초기화
                        print("졸음")
            
#                         if(index_write==0): #쓰기 인덱스 체크
#                             os_dirpath="C://eGovFrame-3.10.0//bin//eclipse//checkupload//vid_test_result" # 텍스트 저장할 폴더경로
#                             os_filepath=os_dirpath+"//test_result.txt" #결과 저장할 텍스트경로
#                             if(os.path.isdir(os_dirpath)!=True): #폴더 경로 체크 . 없으면 생성
#                                 os.mkdir(os_dirpath)
#                             with open(os_filepath, "w") as file:
#                                 file.write(file_path)
#                                 #file.write("World!")
#                             index_write=1
                    # visualize
                    state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
                    state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

                    state_l = state_l % pred_l
                    state_r = state_r % pred_r

                    # 색 지정
                    if pred_l > 0.1:
                        l_color = (255, 255, 255)
                    else:
                        l_color = (0, 0, 255)
                    if pred_r > 0.1:
                        r_color = (255, 255, 255)
                    else:
                        r_color = (0, 0, 255)

                    cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(l_color), thickness=2)
                    cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(r_color), thickness=2)

                    cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (l_color), 2)
                    cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (r_color), 2)

                    #print("판독 실행중")


#                     img = cv2.resize(img, dsize=(640, 480))
#                     cv2.imshow('result', img)
#                     if cv2.waitKey(1) == ord('q'):
#                         cv2.destroyAllWindows()
#                         break

    
    cap.release()       


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 17, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 8, 128)         7385

C:\Users\user\AppData\Local\Temp\ipykernel_6672\3198248789.py:55: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)


1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 15ms/step
졸음
1/1 [==============================] - 0s 15ms/step
졸음
1/1 [==============================] - 0s 15ms/step
졸음
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 13ms/step
졸음
1/1 [==============================] - 0s 15ms/step
졸음
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 12ms/step
졸음
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step
졸음
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 15ms/step
졸음
1/1 [==============================] - 0s 13ms/step
졸음
1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 13ms/step
졸음
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step
졸음
1/1 [==============================] - 0s 17ms/step


KeyboardInterrupt: 

In [ ]:



@app.route('/fileUpload', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        # 저장할 경로 + 파일명
        f.save(secure_filename(f.filename))
        return 'uploads 디렉토리 -> 파일 업로드 성공!'